In [109]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd

class ExtractorTFIDF:
    def __init__(self):
        self.vectorizer = TfidfVectorizer()
        self.feature_names = None

    def extract_tfidf_features(self, sentences, tipos=None, min_words=0):
        if tipos is None:
            tipos = [''] * len(sentences)  # Definindo todos como um tipo padrão se não especificado

        # Filtrar as frases com base no número mínimo de palavras e tipo de frase
        filtered_data = []
        for sentence, tipo in zip(sentences, tipos):
            if len(sentence.split()) >= min_words:
                filtered_data.append((sentence, tipo))

        # Separar frases filtradas por tipo
        grouped_sentences = {}
        for sentence, tipo in filtered_data:
            if tipo not in grouped_sentences:
                grouped_sentences[tipo] = []
            grouped_sentences[tipo].append(sentence)

        # Preparação para armazenar resultados
        all_data = []

        # Iterar sobre cada tipo de frase e extrair as características TF-IDF
        for tipo, sentences in grouped_sentences.items():
            tfidf_matrix = self.vectorizer.fit_transform(sentences)
            self.feature_names = self.vectorizer.get_feature_names_out()

            # Iterar sobre cada frase para encontrar as palavras mais importantes, calcular média e desvio padrão e contar palavras
            for i, sentence in enumerate(sentences):
                tfidf_scores = tfidf_matrix[i].toarray().flatten()
                important_words_indices = tfidf_scores.argsort()[-3:][::-1]  # Top 3 palavras mais importantes
                important_words = [self.feature_names[idx] for idx in important_words_indices]
                pontuacoes = [tfidf_scores[idx] for idx in important_words_indices]

                # Calculando média e desvio padrão das pontuações TF-IDF
                media_tfidf = np.mean(tfidf_scores)
                desvio_padrao_tfidf = np.std(tfidf_scores)

                # Contando palavras na frase
                num_palavras = len(sentence.split())

                # Adicionando informações no formato de dicionário
                all_data.append({
                    'Tipo': tipo,
                    'Frase': sentence,
                    'Palavras_importantes': important_words,
                    'Pontuacoes_TFIDF': pontuacoes,
                    'Media_TFIDF': media_tfidf,
                    'Desvio_Padrao_TFIDF': desvio_padrao_tfidf,
                    'Contagem_Palavras': num_palavras
                })

        # Retornando um DataFrame do Pandas com os dados coletados
        df = pd.DataFrame(all_data)
        return df

    def contar_frequencia_palavras_importantes(self, df, tipo=None):
        frequencia = {}
        if tipo is not None:
            df = df[df['Tipo'] == tipo]

        for palavras_lista in df['Palavras_importantes']:
            for palavra in palavras_lista:
                if palavra in frequencia:
                    frequencia[palavra] += 1
                else:
                    frequencia[palavra] = 1

        df_freq = pd.DataFrame(list(frequencia.items()), columns=['Palavra', 'Frequencia'])
        df_freq = df_freq.sort_values(by='Frequencia', ascending=False).reset_index(drop=True)

        # Adicionando a coluna Tipo ao DataFrame de frequência
        df_freq['Tipo'] = tipo if tipo is not None else 'Todos'

        return df_freq

# Exemplo de uso da classe com entrada do usuário
if __name__ == "__main__":
    frases = [
        "O céu está azul hoje.",
        "A primavera chegou mais cedo este ano.",
        "Gosto de caminhar ao pôr do sol.",
        "A vida é uma jornada cheia de surpresas.",
        "A ciência avança a passos largos.",
        "A música é a linguagem universal."
    ]

    tipos = [
        "Meteorologia",
        "Meteorologia",
        "Meteorologia",
        "Filosofia",
        "Filosofia",
        "Filosofia"
    ]

    # Pedir ao usuário para definir o número mínimo de palavras na frase
    while True:
        try:
            min_words = int(input("Digite o número mínimo de palavras na frase para análise: "))
            break
        except ValueError:
            print("Por favor, digite um número inteiro válido.")

    extractor = ExtractorTFIDF()
    df_resultado = extractor.extract_tfidf_features(frases, tipos, min_words)

    # Exibindo o DataFrame final com as palavras importantes e suas pontuações TF-IDF
    print("DataFrame com as palavras importantes e suas pontuações TF-IDF:")
    print(df_resultado)

    # Inicializar um DataFrame vazio para armazenar as frequências de todas as palavras importantes
    df_final = pd.DataFrame()

    # Contar a frequência das palavras importantes em todas as frases processadas para cada tipo
    for tipo in df_resultado['Tipo'].unique():
        df_tipo = df_resultado[df_resultado['Tipo'] == tipo]
        df_frequencia_palavras = extractor.contar_frequencia_palavras_importantes(df_tipo, tipo)
        print(f"\nDataFrame com a frequência das palavras importantes para o Tipo '{tipo}':")
        print(df_frequencia_palavras)

        # Concatenar o DataFrame de frequência ao DataFrame final
        df_final = pd.concat([df_final, df_frequencia_palavras], ignore_index=True)

    # Exibir o DataFrame final com todas as frequências de palavras importantes para todos os tipos
    print("\nDataFrame final com a frequência das palavras importantes para todos os tipos:")
    print(df_final)



Digite o número mínimo de palavras na frase para análise: 5
DataFrame com as palavras importantes e suas pontuações TF-IDF:
           Tipo                                     Frase  \
0  Meteorologia                     O céu está azul hoje.   
1  Meteorologia    A primavera chegou mais cedo este ano.   
2  Meteorologia          Gosto de caminhar ao pôr do sol.   
3     Filosofia  A vida é uma jornada cheia de surpresas.   
4     Filosofia         A ciência avança a passos largos.   
5     Filosofia         A música é a linguagem universal.   

             Palavras_importantes  \
0               [céu, azul, hoje]   
1          [ano, primavera, mais]   
2                  [sol, pôr, ao]   
3          [vida, uma, surpresas]   
4       [passos, largos, ciência]   
5  [universal, música, linguagem]   

                                    Pontuacoes_TFIDF  Media_TFIDF  \
0                                    [0.5, 0.5, 0.5]     0.117647   
1  [0.4082482904638631, 0.4082482904638631, 0.408.

In [110]:
df_resultado

,Tipo,Frase,Palavras_importantes,Pontuacoes_TFIDF,Media_TFIDF,Desvio_Padrao_TFIDF,Contagem_Palavras
0,Meteorologia,O céu está azul hoje.,"[céu, azul, hoje]","[0.5, 0.5, 0.5]",0.117647,0.212091,5
1,Meteorologia,A primavera chegou mais cedo este ano.,"[ano, primavera, mais]","[0.4082482904638631, 0.4082482904638631, 0.408...",0.144088,0.195096,7
2,Meteorologia,Gosto de caminhar ao pôr do sol.,"[sol, pôr, ao]","[0.37796447300922725, 0.37796447300922725, 0.3...",0.155632,0.186016,7
3,Filosofia,A vida é uma jornada cheia de surpresas.,"[vida, uma, surpresas]","[0.4082482904638631, 0.4082482904638631, 0.408...",0.188422,0.203519,8
4,Filosofia,A ciência avança a passos largos.,"[passos, largos, ciência]","[0.5, 0.5, 0.5]",0.153846,0.230769,6
5,Filosofia,A música é a linguagem universal.,"[universal, música, linguagem]","[0.5773502691896257, 0.5773502691896257, 0.577...",0.133235,0.243252,6


In [112]:
df_final

,Palavra,Frequencia,Tipo
0,céu,1,Meteorologia
1,azul,1,Meteorologia
2,hoje,1,Meteorologia
3,ano,1,Meteorologia
4,primavera,1,Meteorologia
5,mais,1,Meteorologia
6,sol,1,Meteorologia
7,pôr,1,Meteorologia
8,ao,1,Meteorologia
9,vida,1,Filosofia
